# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
#csv_file = os.path.join('..', 'WeatherPy', 'Output', 'ity.csv')
weather_data = "../WeatherPy/output_data/Cities.csv"

# Read the weather data and the study results
read_weather = pd.read_csv(weather_data)

# Combine the data into a single dataset
city_df = pd.DataFrame(read_weather)
city_df.head()

,City ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,Port Elizabeth,-33.9180,25.5701,71.10,55,0,1.14,1627721205,ZA
1,1,Hermanus,-34.4187,19.2345,63.90,64,95,1.23,1627721007,ZA
2,2,Coihaique,-45.5752,-72.0662,31.95,80,0,3.44,1627720929,CL
3,3,Lebu,-37.6167,-73.6500,45.90,86,40,14.63,1627721205,CL
4,4,Souillac,-20.5167,57.5167,75.58,60,20,14.97,1627721027,MU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [19]:
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
fig.add_layer(heat_layer)
plt.savefig("Heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
ideal_df = city_df[(city_df['Max Temperature'] > 70) & (city_df['Max Temperature'] < 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] < 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] == 0]
ideal_df

,City ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,Port Elizabeth,-33.9180,25.5701,71.10,55,0,1.14,1627721205,ZA
25,25,Port Alfred,-33.5906,26.8910,72.27,36,0,8.01,1627721208,ZA
46,46,East London,-33.0153,27.9116,70.63,29,0,3.18,1627721100,ZA
232,232,Richards Bay,-28.7830,32.0377,72.32,48,0,7.76,1627721232,ZA
374,374,Agadir,30.4202,-9.5982,72.05,51,0,3.74,1627721181,MA
503,503,Graaff-Reinet,-32.2522,24.5308,71.85,11,0,9.35,1627721261,ZA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = ideal_df

In [25]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
     # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Port Elizabeth.
Closest hotel is 39 On Nile Guest House.
------------
Retrieving Results for Index 25: Port Alfred.
Closest hotel is The Halyards Hotel.
------------
Retrieving Results for Index 46: East London.
Closest hotel is Tu Casa.
------------
Retrieving Results for Index 232: Richards Bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 374: Agadir.
Closest hotel is Iberostar Founty Beach.
------------
Retrieving Results for Index 503: Graaff-Reinet.
Closest hotel is Rietjiesbos B&B.
------------


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
plt.savefig("Hotel_Map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>